In [ ]:
import sqlite3
import paho.mqtt.client as mqtt
import json

machine_id = "machine_01"
db = sqlite3.connect(f"{machine_id}.db")
cur = db.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS measurements (
    timestamp TEXT,
    step_id TEXT,
    value REAL,
    mean REAL,
    variance REAL
);
""")
db.commit()

def on_message(client, userdata, msg):
    data = json.loads(msg.payload.decode())
    cur.execute("INSERT INTO measurements VALUES (?, ?, ?, ?, ?)", (
        data["timestamp"], data["step_id"], data["value"],
        data["mean"], data["variance"]
    ))
    db.commit()

client = mqtt.Client()
client.on_message = on_message
client.connect("localhost", 1883, 60)
client.subscribe("factory/machine_01/#")
client.loop_forever()
